# library imports

In [192]:
import pandas as pd 
import numpy as np
import os
from datetime import datetime


In [193]:
ratings = {'Rating': [], 'Date': [], 'Resort': []}
for file in os.listdir('comments/'):
    filepath = os.path.join('comments/', file)
    df = pd.read_csv(filepath)
    ratings['Rating'] = ratings['Rating']+df['Rating'].values.tolist()
    ratings['Date'] = ratings['Date']+df['Date'].values.tolist()
    ratings['Resort'] = ratings['Resort']+[file.split('.')[0] for i in range(df.shape[0])]

In [194]:
ratings = pd.DataFrame(ratings)
ratings.head()

,Rating,Date,Resort
0,4,2017-04-22 03:53:09,Charlotte
1,5,2017-06-11 03:54:51,Charlotte
2,5,2017-10-18 07:23:51,Charlotte
3,4,2017-12-04 09:57:32,Charlotte
4,4,2017-12-08 06:26:21,Charlotte


In [195]:
ratings['Resort']=ratings['Resort'].map(lambda x: 'Charlotte Pass' if x=='Charlotte' else x)

In [196]:
def compute_custom_week(date):
    date = datetime.strptime(date,"%Y-%m-%d %H:%M:%S")
    week1_start = pd.to_datetime(f"{date.year}-06-09")
    days_diff = (date - week1_start).days

    week_num = (days_diff // 7) + 1

    return 'week'+str(week_num) if week_num >= 1 and week_num<=15 else None  

In [197]:
compute_custom_week(ratings['Date'].values[0])

In [198]:
ratings['Week'] = ratings['Date'].apply(lambda x:compute_custom_week(x))

In [199]:
ratings = ratings[~ratings['Week'].isnull()]

In [200]:
ratings

,Rating,Date,Resort,Week
1,5,2017-06-11 03:54:51,Charlotte Pass,week1
19,1,2018-08-19 05:10:19,Charlotte Pass,week11
20,1,2018-08-26 00:55:01,Charlotte Pass,week12
35,5,2019-06-09 11:49:33,Charlotte Pass,week1
36,5,2019-06-20 11:50:03,Charlotte Pass,week2
...,...,...,...,...
9559,2,2025-08-27 11:34:04,Thredbo,week12
9560,4,2025-08-27 12:16:21,Thredbo,week12
9561,5,2025-08-29 04:46:27,Thredbo,week12
9562,5,2025-08-29 12:18:15,Thredbo,week12


In [202]:
ratings['Year'] = ratings['Date'].apply(lambda x: datetime.strptime(x,"%Y-%m-%d %H:%M:%S").year)

In [213]:
ratings = ratings.groupby(['Resort','Week', 'Year'])['Rating'].mean()

In [214]:
ratings.to_csv('ratings.csv')